In [16]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from imblearn.over_sampling import SMOTE
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import lime
from lime.lime_tabular import LimeTabularExplainer

# Data Loading with Augmentation
base_path = '../Database/'
data, labels = [], []

data_gen = ImageDataGenerator(
    rotation_range=30, width_shift_range=0.2, height_shift_range=0.2,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True,
    brightness_range=[0.8, 1.2], fill_mode='nearest'
)

# Load Data
def load_images_from_folder(folder, label):
    folder_path = os.path.join(base_path, folder)
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        img = cv2.resize(img, (128, 128))
        data.append(img)
        labels.append(label)

load_images_from_folder("Normal", "Normal")
for folder in ["Lung_Opacity", "Viral Pneumonia"]:
    load_images_from_folder(folder, "Lung_Disease")

data = np.array(data).astype('float32') / 255.0
labels = np.array(labels)

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    data, labels_encoded, test_size=0.2, random_state=42, stratify=labels_encoded
)

# Extract HOG Features
def extract_hog_features(images):
    return np.array([hog(img, orientations=9, pixels_per_cell=(8, 8),
                          cells_per_block=(2, 2), visualize=False) for img in images])

X_train_hog = extract_hog_features(X_train)
X_test_hog = extract_hog_features(X_test)

# CNN Feature Extraction (ResNet50)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

x = GlobalAveragePooling2D()(base_model.output)
x = BatchNormalization()(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.5)(x)

feature_extractor = Model(inputs=base_model.input, outputs=x)

def extract_cnn_features(images):
    images = np.repeat(images[..., np.newaxis], 3, axis=-1)
    return feature_extractor.predict(images)

X_train_cnn = extract_cnn_features(X_train)
X_test_cnn = extract_cnn_features(X_test)

# Combine CNN + HOG Features
X_train_features = np.hstack((X_train_cnn, X_train_hog))
X_test_features = np.hstack((X_test_cnn, X_test_hog))

# Handle Imbalance Using SMOTE
smote = SMOTE(random_state=42)
X_train_features, y_train = smote.fit_resample(X_train_features, y_train)

# Feature Scaling
scaler = StandardScaler()
X_train_features = scaler.fit_transform(X_train_features)
X_test_features = scaler.transform(X_test_features)

# Model Training
model = tf.keras.Sequential([
    Dense(256, activation='relu', kernel_regularizer=l2(0.01), input_shape=(X_train_features.shape[1],)),
    Dropout(0.5),
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Learning Rate Scheduler & Early Stopping
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train_features, y_train, validation_data=(X_test_features, y_test),
                    epochs=50, callbacks=[lr_scheduler, early_stopping])

# Evaluation
y_pred = (model.predict(X_test_features) > 0.5).astype("int32")
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

# LIME Explanation
explainer = LimeTabularExplainer(X_train_features, feature_names=["Feature " + str(i) for i in range(X_train_features.shape[1])], class_names=label_encoder.classes_, discretize_continuous=True)
exp = explainer.explain_instance(X_test_features[0], lambda x: model.predict(x), num_features=10)
exp.show_in_notebook()

# Plot Training & Validation Graphs
def plot_training_history(history):
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy', color='blue')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='red')
    plt.title('Model Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss', color='blue')
    plt.plot(history.history['val_loss'], label='Validation Loss', color='red')
    plt.title('Model Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.show()

plot_training_history(history)


83/83 ━━━━━━━━━━━━━━━━━━━━ 82s 954ms/step
21/21 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step 
Epoch 1/50


C:\Users\thebe\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.7627 - loss: 7.2611 - val_accuracy: 0.8782 - val_loss: 5.2629 - learning_rate: 0.0010
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.8672 - loss: 4.7790 - val_accuracy: 0.8647 - val_loss: 3.2884 - learning_rate: 0.0010
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.8918 - loss: 2.9257 - val_accuracy: 0.8632 - val_loss: 2.1215 - learning_rate: 0.0010
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.9292 - loss: 1.8522 - val_accuracy: 0.8722 - val_loss: 1.5712 - learning_rate: 0.0010
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.9265 - loss: 1.3815 - val_accuracy: 0.8782 - val_loss: 1.2980 - learning_rate: 0.0010
Epoch 6/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.9352 - loss: 1.1236 - val_accuracy: 0.8827 - val_loss: 1.1691 - learning_rate: 0.0010
Epoch 7/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.9265 - loss: 1.0209 - val_

C:\Users\thebe\AppData\Local\Programs\Python\Python312\Lib\site-packages\lime\lime_tabular.py:372: UserWarning: 
                    Prediction probabilties do not sum to 1, and
                    thus does not constitute a probability space.
                    Check that you classifier outputs probabilities
                    (Not log probabilities, or actual class predictions).
                    
  warnings.warn("""


IndexError: index 1 is out of bounds for axis 1 with size 1